In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh
!pip install streamlit pyngrok ollama firebase-admin

In [ ]:
import os
env = os.environ.copy()
env["OLLAMA_HOST"] = "0.0.0.0"
env["OLLAMA_ORIGINS"] = "*"

import subprocess
def run_ollama_serve():
  subprocess.Popen(["ollama", "serve"], env=env)

import threading
thread = threading.Thread(target=run_ollama_serve)
thread.start()

import time
time.sleep(5)

In [ ]:
!ollama pull mistral

print("creating cloudflare tunnel...")

# Cài Cloudflared
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O /usr/local/bin/cloudflared 2>/dev/null || echo "Cloudflared đã có"
!chmod +x /usr/local/bin/cloudflared

# Dừng cloudflared cũ
!pkill -9 cloudflared 2>/dev/null

import subprocess
import time
import re

# Khởi chạy Cloudflare Tunnel
process = subprocess.Popen(
    ['nohup', 'cloudflared', 'tunnel', '--url', 'http://localhost:11434'],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True
)

# Đợi lấy URL
time.sleep(12)
ollama_url = None

for i in range(30):
    line = process.stdout.readline()
    if 'trycloudflare.com' in line:
        match = re.search(r'https://[^\s]+trycloudflare\.com', line)
        if match:
            ollama_url = match.group(0)
            break
    time.sleep(0.5)

if ollama_url:
    print(f"\nURL:\n{ollama_url}\n")

    # Test
    import requests
    try:
        test = requests.get(f"{ollama_url}/api/tags", timeout=15)
        if test.status_code == 200:
            print("\nTunnel active!")
            print(f"Models: {test.json().get('models', [])}")
        else:
            print(f"\nTunnel trả về: {test.status_code}")
    except Exception as e:
        print(f"\nTest lỗi: {e}")
else:
    print("\nKhông lấy được URL")

In [ ]:
%%writefile app.py
import streamlit as st
import requests
import firebase_admin
from firebase_admin import credentials, auth, firestore
import os

OLLAMA_URL = "https://placed-mods-packets-holding.trycloudflare.com"

FIREBASE_API_KEY = "your_firebase_key"


# SESSION STATE & FIREBASE
if 'user_logged_in' not in st.session_state:
    st.session_state.user_logged_in = False
if 'db' not in st.session_state:
    st.session_state.db = None
if 'user_id' not in st.session_state:
    st.session_state.user_id = None
if 'user_email' not in st.session_state:
    st.session_state.user_email = None

def init_firebase():
    if not st.session_state.db:
        try:
            if not firebase_admin._apps:
                # Đảm bảo bạn đã tải file 'firebase.json' lên Colab
                cred = credentials.Certificate("firebase.json")
                firebase_admin.initialize_app(cred)
            st.session_state.db = firestore.client()
            return True
        except Exception as e:
            st.error(f"Firebase lỗi: {e}")
            st.caption("Lỗi thường gặp: Thiếu file 'firebase.json' hoặc file không hợp lệ.")
            return False
    return True

def authenticate_user(email, password, is_register=False):
    if not init_firebase():
        return
    try:
        if is_register:
            auth.create_user(email=email, password=password)
            st.success("Đăng ký thành công! Vui lòng đăng nhập.")
        else:
            url = f"https://identitytoolkit.googleapis.com/v1/accounts:signInWithPassword?key={FIREBASE_API_KEY}"
            response = requests.post(url, json={
                "email": email, "password": password, "returnSecureToken": True
            })
            if response.status_code == 200:
                data = response.json()
                st.session_state.user_id = data['localId']
                st.session_state.user_email = email
                st.session_state.user_logged_in = True
                st.success(f"Chào {email}!")
                st.rerun()
            else:
                st.error("Sai email/password")
    except Exception as e:
        st.error(f"Lỗi xác thực: {e}")

# HÀM LỊCH TRÌNH
def generate_itinerary(origin, dest, num_days, interests, pace, ollama_url):
    interest_str = ", ".join(interests) if interests else "general sightseeing"
    prompt = f"""Tạo tour du lịch từ {origin} đến {dest} trong {num_days}.

INTEREST: {interest_str}
PACE: {pace}

REQUEST:
1. Chia theo từng ngày (Sáng/Chiều/Tối)
2. Gợi ý địa điểm cụ thể tại {dest}
3. Thêm mẹo thực tế (giá vé, thời gian, lưu ý)
4. Viết bằng tiếng Việt

FORMAT:
**Ngày 1:**
- **Sáng (7:00-11:00):** Tham quan [Địa điểm]. Mẹo: ...
- **Chiều (14:00-18:00):** ...
- **Tối (19:00-22:00):** ...

Hãy tạo lịch trình ngay:"""
    try:
        # Kiểm tra kết nối
        test_response = requests.get(f"{ollama_url}/api/tags", timeout=10)
        if test_response.status_code != 200:
            return f"Kết nối Ollama thất bại: {test_response.status_code} - {test_response.text[:200]}"

        response = requests.post(
            f"{ollama_url}/api/generate",
            json={
                "model": "mistral", "prompt": prompt, "stream": False,
                "options": {"temperature": 0.7, "num_predict": 1000}
            },
            headers={"Content-Type": "application/json"}, timeout=120
        )
        if response.status_code == 200:
            return response.json().get('response', 'Không có phản hồi')
        else:
            return f"Lỗi {response.status_code}: {response.text[:200]}"
    except requests.exceptions.ConnectionError:
        return "Mất kết nối tới Ollama. Kiểm tra lại URL tunnel hoặc khởi động lại Ollama."
    except requests.exceptions.Timeout:
        return "Timeout khi kết nối Ollama. Thử lại sau."
    except Exception as e:
        return f"Lỗi không xác định: {str(e)}"

# GIAO DIỆN
st.set_page_config(page_title="DEMO MINI TRAVEL", layout="wide")
st.title("Demo mini travel")

with st.sidebar:
    st.subheader("System Status")
    if "your-tunnel-name" not in OLLAMA_URL:
        try:
            test = requests.get(f"{OLLAMA_URL}/api/tags", timeout=5)
            if test.status_code == 200:
                st.success("Ollama Connected")
                st.caption(f"URL: {OLLAMA_URL[:40]}...")
            else:
                st.error(f"Ollama Error: {test.status_code}")
        except:
            st.warning("Không thể kiểm tra Ollama. URL có thể sai.")
    else:
        st.warning("cập nhật OLLAMA_URL.")

if not st.session_state.user_logged_in:
    st.subheader("Đăng nhập/Đăng ký")
    with st.form("login_form"):
        email = st.text_input("Email")
        password = st.text_input("Password", type="password")
        col1, col2 = st.columns(2)
        with col1:
            login = st.form_submit_button("Đăng nhập", use_container_width=True)
        with col2:
            register = st.form_submit_button("Đăng ký", use_container_width=True)
    if login and email and password:
        authenticate_user(email, password)
    if register and email and password:
        authenticate_user(email, password, is_register=True)
else:
    init_firebase()
    st.sidebar.subheader("User")
    st.sidebar.success(f"Logged in as: {st.session_state.user_email}")
    if st.sidebar.button("Đăng xuất", use_container_width=True):
        for key in list(st.session_state.keys()):
            del st.session_state[key]
        st.rerun()

    st.divider()
    tab1, tab2 = st.tabs(["PLANNER", "HISTORY"])

    with tab1:
        st.header("Create new itinerary")
        with st.form("travel_form"):
            col1, col2, col3 = st.columns(3)
            with col1:
                origin = st.text_input("Origin", "Hồ Chí Minh")
                dest = st.text_input("Destination", "Cà Mau")
            with col2:
                num_days = st.number_input("Days", min_value=1, max_value=14, value=3, step=1)
                pace = st.selectbox("Pace", ["Relaxed", "Normal", "Tight"])
            with col3:
                interests = st.multiselect(
                    "Interests",
                    ['Food', 'Nature', 'Shopping', 'Adventure'],
                    default=['Food', 'Nature']
                )
            submitted = st.form_submit_button("Create", use_container_width=True, type="primary")

        if submitted and dest and num_days > 0:
            with st.spinner('AI đang lên lịch trình...'):
                itinerary = generate_itinerary(origin, dest, num_days, interests, pace, OLLAMA_URL)
                if "Lỗi" in itinerary or "Mất kết nối" in itinerary or "Kết nối thất bại" in itinerary:
                    st.error(itinerary)
                else:
                    st.success(f"**{num_days} ngày** khám phá từ **{origin}** đến **{dest}**")
                    st.divider()
                    st.markdown(itinerary)
                    if st.session_state.db:
                        try:
                            st.session_state.db.collection('itineraries').add({
                                'user_id': st.session_state.user_id, 'destination': dest,
                                'input': {'origin': origin, 'num_days': num_days, 'interests': interests, 'pace': pace},
                                'itinerary': itinerary, 'timestamp': firestore.SERVER_TIMESTAMP
                            })
                        except Exception as e:
                            st.warning(f"Không lưu được vào lịch sử: {e}")
    with tab2:
        st.header("History")
        if st.session_state.db:
            try:
                docs = st.session_state.db.collection('itineraries') \
                    .where('user_id', '==', st.session_state.user_id) \
                    .order_by('timestamp', direction=firestore.Query.DESCENDING) \
                    .limit(20).stream()
                items = list(docs)
                if not items:
                    st.info("NONE.")
                else:
                    for i, doc in enumerate(items, 1):
                        data = doc.to_dict()
                        inp = data.get('input', {})
                        time_str = data.get('timestamp').strftime('%Y-%m-%d %H:%M') if data.get('timestamp') else 'N/A'
                        with st.expander(f"#{i} - {data.get('destination', 'N/A')} ({inp.get('num_days', 'N/A')} ngày) - {time_str}"):
                            st.markdown(f"**From:** {inp.get('origin', 'N/A')}")
                            st.markdown(f"**Pace:** {inp.get('pace', 'N/A')}")
                            st.markdown(f"**Interests:** {', '.join(inp.get('interests', []))}")
                            st.divider()
                            st.markdown(data.get('itinerary', 'N/A'))
            except Exception as e:
                st.error(f"Lỗi tải lịch sử: {e}")

In [6]:
# Dừng Streamlit cũ
!pkill -9 streamlit 2>/dev/null

import time
time.sleep(3)

# Khởi động Streamlit
!streamlit run app.py &>/dev/null&

time.sleep(5)

# Tạo Cloudflare Tunnel cho Streamlit
import subprocess
import re

cf_process = subprocess.Popen(
    ['cloudflared', 'tunnel', '--url', 'http://localhost:8501'],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True
)

time.sleep(12)
streamlit_url = None

for i in range(30):
    line = cf_process.stdout.readline()
    if 'trycloudflare.com' in line:
        match = re.search(r'https://[^\s]+trycloudflare\.com', line)
        if match:
            streamlit_url = match.group(0)
            break
    time.sleep(0.5)

if streamlit_url:
    print("STREAMLIT ĐÃ KHỞI ĐỘNG!")
    print(f"\nURL:\n{streamlit_url}\n")
else:
    print("Lỗi")

STREAMLIT ĐÃ KHỞI ĐỘNG!

URL:
https://admission-opinions-consecutive-aquarium.trycloudflare.com

